<a href="https://colab.research.google.com/github/AbdollahMasoud/IJCNN-2023/blob/main/IJCNN_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Get Data from Repo
!git clone 'https://github.com/AbdollahMasoud/IJCNN-2023'

Cloning into 'IJCNN-2023'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), 11.70 MiB | 1.40 MiB/s, done.


In [15]:
# Extract Image Folders
!unzip 'IJCNN-2023/Dataset.zip' -d '/content/'

Archive:  IJCNN-2023/Dataset.zip
replace /content/FR0/J000001.57-092940.3.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
# Adding Preliminaries

!pip install catboost

import gc
import os
import cv2
import time
import imutils
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from imutils import paths
from statistics import mean 
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from datetime import datetime as dt
from sklearn.decomposition import PCA
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.0 MB/s eta 0:00:00


In [10]:
# Reading FRI Images
imagePaths = list(paths.list_images(r"/content/FRI"))

data = []
labels = []

for imagePath in imagePaths:
  image = cv2.imread(imagePath,cv2.IMREAD_GRAYSCALE)
  image = image[124:174,124:174]
  data.append(image)
  label = 1
  labels.append(label)
print("Loaded FRI...")

Loaded FRI...


In [11]:
# Reading FRII Images
imagePaths = list(paths.list_images(r"/content/FRII"))

for imagePath in imagePaths:
  image = cv2.imread(imagePath,cv2.IMREAD_GRAYSCALE)
  image = image[124:174,124:174]
  data.append(image)
  label = 2
  labels.append(label)
print("Loaded FRII...")

Loaded FRII...


In [12]:
# Reading FR0 Images  
imagePaths = list(paths.list_images(r"/content/FR0"))

for imagePath in imagePaths:
  image = cv2.imread(imagePath,cv2.IMREAD_GRAYSCALE)
  image = image[124:174,124:174]
  data.append(image)
  label = 0
  labels.append(label)
print("Loaded FR0...")

Loaded FR0...


In [17]:
labels = np.array(labels,dtype='uint8')
data = np.array(data,dtype='uint8')

# Flatten Images
data = data.reshape((len(data),50*50))
data = pd.DataFrame(data)
labels = pd.DataFrame(labels)
BB = pd.concat([data,labels], axis=1)
FRI = BB[0:5008]
FRII = BB[5008:7074]
FR0 = BB[7074::] 

In [ ]:

# Choose Number of Samples per Class ; Max is 2000 for a Balanced Dataset
samplesize=2000

# Balancing the Dataset
FRI_s = FRI.sample(n=samplesize,random_state=42)
FRII_s = FRII.sample(n=samplesize,random_state=42)
FR0_s = FR0.sample(n=samplesize,random_state=42)
Images_s = pd.concat([FRI_s,FRII_s,FR0_s])
Images_s = shuffle(Images_s,random_state=42)
lbl = Images_s.iloc[:,-1:]
Imgs = Images_s.iloc[:,:-1]

# Using PCA
pca_fit = PCA(n_components=34)
Imgs=pca_fit.fit_transform(Imgs)
Imgs = np.array(Imgs)
lbl = np.array(lbl)

ConfMat_CAT = []
TTrain_CAT = []
TTest_CAT = []
ConfMat_LGB = []
TTrain_LGB = []
TTest_LGB = []
ConfMat_XGB = []
TTrain_XGB = []
TTest_XGB = []

# Setting up Repeated K Fold
kf = RepeatedKFold(n_splits=5,n_repeats=10,random_state=42)

count=0

for train_index, test_index in kf.split(Imgs):
  trainX, testX = Imgs[train_index], Imgs[test_index]
  trainY, testY = lbl[train_index], lbl[test_index]

  trainX = np.array(trainX)
  testX = np.array(testX)
  trainY = np.array(trainY)
  testY = np.array(testY)
  
  # Train & Test CatBoost Classifier
  # boosting_type='Ordered' is slow ; For faster execution consider using boosting_type='Plain'
  clf = ctb.CatBoostClassifier(logging_level='Silent',classes_count=3,boosting_type='Ordered',learning_rate=0.1,l2_leaf_reg=3,max_depth=6,loss_function='MultiClass')
  start = dt.now()
  clf.fit(trainX,trainY.ravel())
  time = (dt.now() - start).total_seconds()
  TTrain_CAT.append(time)
  start = dt.now()
  predY=clf.predict(testX)
  time = (dt.now() - start).total_seconds()
  TTest_CAT.append(time)
  ConfMat=confusion_matrix(testY,predY)
  ConfMat_CAT.append(ConfMat)

  # Train & Test LightGBM Classifier
  clf = lgb.LGBMClassifier(objective='multiclass',num_class=3)
  start = dt.now()
  clf.fit(trainX,trainY.ravel())
  time = (dt.now() - start).total_seconds()
  TTrain_LGB.append(time)
  start = dt.now()
  predY=clf.predict(testX)
  time = (dt.now() - start).total_seconds()
  TTest_LGB.append(time)
  ConfMat=confusion_matrix(testY,predY)
  ConfMat_LGB.append(ConfMat)

  # Train & Test XGB Classifier
  clf = xgb.XGBClassifier(booster='gbtree',objective='multi:softmax',num_class=3,learning_rate=0.3,gamma=0,max_depth=9)
  start = dt.now()
  clf.fit(trainX,trainY.ravel())
  time = (dt.now() - start).total_seconds()
  TTrain_XGB.append(time)
  start = dt.now()
  predY=clf.predict(testX)
  time = (dt.now() - start).total_seconds()
  TTest_XGB.append(time)
  ConfMat=confusion_matrix(testY,predY)
  ConfMat_XGB.append(ConfMat)

  count+=1
  print(count,'/','50 Iterations Done')

1 / 50 Iterations Done


In [ ]:
# Plotting and Displaying Results

CAT=np.sum(ConfMat_CAT,axis = 0)
print(CAT)
disp = ConfusionMatrixDisplay(confusion_matrix=CAT, display_labels=clf.classes_)
disp.plot()
plt.show()
APR=CAT
CAT_acc=(APR[0,0]+APR[1,1]+APR[2,2])/np.sum(APR)
CAT_FR0_P=APR[0,0]/sum(APR[:,0])
CAT_FR1_P=APR[1,1]/sum(APR[:,1])
CAT_FR2_P=APR[2,2]/sum(APR[:,2])
CAT_FR0_R=APR[0,0]/sum(APR[0,:])
CAT_FR1_R=APR[1,1]/sum(APR[1,:])
CAT_FR2_R=APR[2,2]/sum(APR[2,:])
CAT_FR0_F1=2*(CAT_FR0_P*CAT_FR0_R)/(CAT_FR0_P+CAT_FR0_R)
CAT_FR1_F1=2*(CAT_FR1_P*CAT_FR1_R)/(CAT_FR1_P+CAT_FR1_R)
CAT_FR2_F1=2*(CAT_FR2_P*CAT_FR2_R)/(CAT_FR2_P+CAT_FR2_R)
print("CAT Accuracy: " + str(CAT_acc))
print("CAT Precision FR0: " + str(CAT_FR0_P))
print("CAT Precision FRI: " + str(CAT_FR1_P))
print("CAT Precision FRII: " + str(CAT_FR2_P))
print("CAT Recall FR0: " + str(CAT_FR0_R))
print("CAT Recall FRI: " + str(CAT_FR1_R))
print("CAT Recall FRII: " + str(CAT_FR2_R))
print("CAT F1 FR0: " + str(CAT_FR0_F1))
print("CAT F1 FRI: " + str(CAT_FR1_F1))
print("CAT F1 FRII: " + str(CAT_FR2_F1))
print("CAT Training Time: " + str(mean(TTrain_CAT)))
print("CAT Testing Time: " + str(mean(TTest_CAT)))

LGB=np.sum(ConfMat_LGB,axis = 0)
print(LGB)
disp = ConfusionMatrixDisplay(confusion_matrix=LGB, display_labels=clf.classes_)
disp.plot()
plt.show()
APR=LGB
LGB_acc=(APR[0,0]+APR[1,1]+APR[2,2])/np.sum(APR)
LGB_FR0_P=APR[0,0]/sum(APR[:,0])
LGB_FR1_P=APR[1,1]/sum(APR[:,1])
LGB_FR2_P=APR[2,2]/sum(APR[:,2])
LGB_FR0_R=APR[0,0]/sum(APR[0,:])
LGB_FR1_R=APR[1,1]/sum(APR[1,:])
LGB_FR2_R=APR[2,2]/sum(APR[2,:])
LGB_FR0_F1=2*(LGB_FR0_P*LGB_FR0_R)/(LGB_FR0_P+LGB_FR0_R)
LGB_FR1_F1=2*(LGB_FR1_P*LGB_FR1_R)/(LGB_FR1_P+LGB_FR1_R)
LGB_FR2_F1=2*(LGB_FR2_P*LGB_FR2_R)/(LGB_FR2_P+LGB_FR2_R)
print("LGB Accuracy: " + str(LGB_acc))
print("LGB Precision FR0: " + str(LGB_FR0_P))
print("LGB Precision FRI: " + str(LGB_FR1_P))
print("LGB Precision FRII: " + str(LGB_FR2_P))
print("LGB Recall FR0: " + str(LGB_FR0_R))
print("LGB Recall FRI: " + str(LGB_FR1_R))
print("LGB Recall FRII: " + str(LGB_FR2_R))
print("LGB F1 FR0: " + str(LGB_FR0_F1))
print("LGB F1 FRI: " + str(LGB_FR1_F1))
print("LGB F1 FRII: " + str(LGB_FR2_F1))
print("LGB Training Time: " + str(mean(TTrain_LGB)))
print("LGB Testing Time: " + str(mean(TTest_LGB)))

XGB=np.sum(ConfMat_XGB,axis = 0)
print(XGB)
disp = ConfusionMatrixDisplay(confusion_matrix=XGB, display_labels=clf.classes_)
disp.plot()
plt.show()
APR=XGB
XGB_acc=(APR[0,0]+APR[1,1]+APR[2,2])/np.sum(APR)
XGB_FR0_P=APR[0,0]/sum(APR[:,0])
XGB_FR1_P=APR[1,1]/sum(APR[:,1])
XGB_FR2_P=APR[2,2]/sum(APR[:,2])
XGB_FR0_R=APR[0,0]/sum(APR[0,:])
XGB_FR1_R=APR[1,1]/sum(APR[1,:])
XGB_FR2_R=APR[2,2]/sum(APR[2,:])
XGB_FR0_F1=2*(XGB_FR0_P*XGB_FR0_R)/(XGB_FR0_P+XGB_FR0_R)
XGB_FR1_F1=2*(XGB_FR1_P*XGB_FR1_R)/(XGB_FR1_P+XGB_FR1_R)
XGB_FR2_F1=2*(XGB_FR2_P*XGB_FR2_R)/(XGB_FR2_P+XGB_FR2_R)
print("XGB Accuracy: " + str(XGB_acc))
print("XGB Precision FR0: " + str(XGB_FR0_P))
print("XGB Precision FRI: " + str(XGB_FR1_P))
print("XGB Precision FRII: " + str(XGB_FR2_P))
print("XGB Recall FR0: " + str(XGB_FR0_R))
print("XGB Recall FRI: " + str(XGB_FR1_R))
print("XGB Recall FRII: " + str(XGB_FR2_R))
print("XGB F1 FR0: " + str(XGB_FR0_F1))
print("XGB F1 FRI: " + str(XGB_FR1_F1))
print("XGB F1 FRII: " + str(XGB_FR2_F1))
print("XGB Training Time: " + str(mean(TTrain_XGB)))
print("XGB Testing Time: " + str(mean(TTest_XGB)))